# Introduction to Parallel Computing with Python

## 1. Party Like it's 1969: fork

One of the oldest and simplest (though far from easiest) ways to run multiple processes on a computer is by using the POSIX interface `fork`.  What `fork` does is create an identical copy of the program currently running, memory and all, and continues execution of both the original and the new process from that point on until the processes are joined back together or killed.  Let's see what this looks like in Python. 

In [94]:
import os # Access to the `fork` command

After using `fork`, we need some method of keeping track of which process we're working with.  The `os` module offers a nice function for finding the ID of a process.  Using `os.getpid`, we can get the ID number of the current process we're working in.  This will return a different value for each process, with a value of 0 for the *parent* process.  That is, the process that spawned the new process.

In Jupyter notebooks, we have to be a little bit careful since the default process actually doesn't have a PID of 0, so we'll first hold onto the PID of the original process.

In [6]:
parent = os.getpid()

print("Parent", parent)

os.fork() # Create a new process

print("Hello From", os.getpid())


Parent 2192640
Hello From 2192640
Hello From 2195140


So let's do a real example that could actually compute something kind of useful.  We'll compute the factorial of 2 different numbers.

In [133]:
data = [5, 10] # Here's some data that get's copied between processes
results = [0, 0] # We'll use this to store the outputs

os.fork()

# Chose which data we act on depending on the PID
if os.getpid() == parent:
    index = 0
else:
    index = 1
    
# Compute the factorial
factorial = 1
for i in range(1, data[index]+1):
    factorial *= i
    
# Store the results in our array
results[index] = factorial
    
print(f'{data[index]}!={factorial}')

5!=120
10!=3628800


In [111]:
print(results)

[120, 0]


What happened?  Since each process only has access to a *copy* of the parent's memory, modification to the memory from another process won't affect the memory of the parent.  So in general more complicated message passing interfaces are generally required.  But this is Python, and we never do anything ourselves, so let's make life easier.

## 2. Basic Parallelism in Native Python

### 2.1 Threads vs Processes

Let's take a step back for a second to do a little theory.  There are two primary ways that parallelism is done (at the high level at least).  We just saw an example using *processes*, which are essentially entirely seperate programs that we launch by making a copy of our current program.  The second is threads, which unlike processes, share the same memory space as their parents and don't create entire copies of the program, but instead run together in parallel within a single process.  What this generally means is that threads tend to be much more light-weight and communication between threads is much simpler.  In Python, since we're not copying the entire global scope when making threads, we are less likely to run into issues when using complicated classes or functions because we don't have to worry about making sure they copy correctly.

Threads do have their disadvantage though.  Since they share the same memory, you must be careful when modifying variables as to not have two threads trying to use and/or modify the same resources at the same time.  This usually is avoided by using a complex system of *barriers*, *locks*, *semaphores*, etc.  Using processes avoids this issue as they all have their own world they live in and don't bother each other.  Another disadvantage of threads is the infamous GIL.

The *Global Interpreter Lock* is a special *lock* object that the Python interpreter itself uses.  What it does is prevent the Python interpreter from interpreting more than one line at a time.  This is because the Python interpreter is not "thread-safe", i.e. it will run into some of the issues mentioned above under some circumstances.  There's a whole history behind the GIL, and a lot of frustrated people.  I would recommend reading up on it a bit simply because it's interesting.  For this notebook, all we need to know though is that it significantly impacts the performance of multithreading in Python.  This performance impact can be significantly reduced if the code spends a lot of time doing things outside of Python.  A few examples being `numpy` functions which are typically written in C or FORTRAN, and device access such as file reading or downloading.

### 2.2 Threads in Python

Python offers a nice and easy interface to creating and running threads through the `threading` module.  In particular, it exposes a `Thread` object that can be used to launch functions in another thread with ease and then manage threads before bringing them all back together.  We'll call these threads that are working on computing some function "workers".

While often times the `Thread` class is subclassed for more complicated programs, we can also invoke it directly by giving it a function and some arguments.  

In [112]:
import threading

def compute_something(N):
    '''Some function that does some kind of long computation
    Setting N lets us change how long it takes'''
    for i in range(N):
        i += 1
    

First, as a benchmark, we'll look at how long it takes to run 10 loops of `compute_something` with 1000000 iterations.

In [113]:
%%timeit

NUM_LOOPS = 10
N = 1000000

for _ in range(NUM_LOOPS):
    compute_something(N)

415 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now we'll try running it in parallel using threads and see if we see any improvement.

In [114]:
%%timeit

NUM_THREADS = 10
N = 1000000

# Make the threads by calling threading.Thread
workers = [threading.Thread(target=compute_something, 
                            args=(N,)) 
           for i in range(NUM_THREADS)]

for worker in workers: # Start each thread
    worker.start()
    
for worker in workers: # Join the threads back together (wait until they finish)
    worker.join()
    

390 ms ± 7.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Wait a second, this took about the same amount of time to run compared to just running it in a loop!  What's going on here?  The problem is that the GIL still won't execute multiple lines simultaniously, so while the order in which each operation is computed may not be sequential, we still have to wait for each line of Python code to finish executing before moving onto the next one.  So not only are we not getting the benefit of parallelism, we may actually be running even slower because of the overhead of setting up the threads!  This is where multiprocessing begins to shine, as it is not limited by the GIL.

### 2.3 Multiprocessing in Python

Python offers a nice and convenient module for `multiprocessing` with not only similar syntax to the `threading` library, but an extended interface to make parallelism easier on the developer.  First we'll just do an example exactly the same as before, but this time instead of using `Thread`, we'll use `Process`. 

In [115]:
import multiprocessing

In [116]:
%%timeit

NUM_PROCS = 10
N = 1000000

# Make the threads by calling multiprocessing.Process
workers = [multiprocessing.Process(target=compute_something, 
                                   args=(N,)) 
           for i in range(NUM_PROCS)]

for worker in workers: # Start each process
    worker.start()
    
for worker in workers: # Join the processes back together
    worker.join()

73.3 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now we're starting to see some improvements!  You'll notice our code still isn't running 10 times faster though.  This is because instantiating new processes has significant overhead since you still need to copy all the memory of the parent process.  This overhead is significantly larger than the overhead for declaring threads, so make sure to keep that in mind.  

### 2.3 Pools

You'll notice that there's still a lot of boiler plate code to make our functions run in parallel.  We have to instantiate workers, start their execution, and we need to join them back to the calling thread.  Worst of all, if we want to get the outputs of the functions, we need to declare extra data strutures to store them in for later, which gets especially complicated with processes.  The `multiprocessing` module offers a more limited interface that can simplify parallelism a little bit and will help us get in the right mindset for more modern Python parallelism techniques and APIs.

Process pools and thread pools are objects that we can declare that help manage workers for us.  These objects allow us to submit a set of *tasks* to the pool and Python will figure out the starting and joining of the workers.  The interface also allows us to easily retrieve the results of a task.  Let's do an example with computing some Fibonacci numbers.

In [117]:
import time # To slow things down

def fibonacci(n):
    time.sleep(1) # Slow down our process
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return n, a

Pools are very easy to instantiate.  In the most basic form, they simply take a single argument being the number of processes you would like the pool to have access to.  They a few methods to facilitate running functions in parallel.  Here we use the `Pool.map` method, which behaves just like the `map` builtin function (though it returns a list instead of an iterator like in Python 2).  The `Pool.map` method takes a function and an iterable of arguments and computes them in sperate processes.  If the number of arguments is larger than the number of processes, it will wait for a process to to be avaible and run the task as soon as it can.  This is very useful for making sure your code is using a consistent amount of resources.

In [120]:
NUM_PROCS = 10
N_VALUES = 20 # Notice we have more values than processes!
process_pool = multiprocessing.Pool(NUM_PROCS)

for n, f in process_pool.map(fibonacci, range(N_VALUES)):
    print(f'fibonacci({n})={f}')

fibonacci(0)=0
fibonacci(1)=1
fibonacci(2)=1
fibonacci(3)=2
fibonacci(4)=3
fibonacci(5)=5
fibonacci(6)=8
fibonacci(7)=13
fibonacci(8)=21
fibonacci(9)=34
fibonacci(10)=55
fibonacci(11)=89
fibonacci(12)=144
fibonacci(13)=233
fibonacci(14)=377
fibonacci(15)=610
fibonacci(16)=987
fibonacci(17)=1597
fibonacci(18)=2584
fibonacci(19)=4181


See how much easier that was?  And we are able to get the results back from each task.  Using `Pool.map` always returns an ordered list, so the iteration won't begin until all of the tasks have completed.  Alternatively there's a method, `Pool.imap`, that returns an iterator and another method, `Pool.imap_async`, which returns an iterator that yields values as soon as they're ready.  The latter method will not maintain the ordering of the original arguments.

Thread pools also exist in the `multiprocessing` module, but they aren't found on the surface.  You need to poke around in one of the submodules to find it.  `ThreadPool` objects found in the `mutliprocessing` module simply shouldn't be used because we have a more modern API in Python to work with them, which brings us to our next topic.

## 3. Modern Parallelism in Native Python

Python 3.2 introduced the `concurrent` module for single-threaded, multi-threaded, and multi-process concurrency.  The module `concurrent.futures` impliments a new way of writing concurrent code in Python using objects called `future` objects which are representative of some task that has either yet to take place or is still currently running.  These are similar to the "workers" we made above but with a much improved interface and don't necessarily correspond to a specific thread or process.  The library also introduces the concept of *executors*, which are similar to pools in that they manage the parallelism for us.  But unlike pools, they offer much greater control and flexibility with how we write and think about our code.  This library interfaces nicely with the new `async`/`await` syntax introduced in Python 3 but that will be a topic for another meeting.  The API and its ideas are used extensively in many modern dsitributed computing libraries for Python.

### 3.1 Executors

Before jump into the code, I'll introduce the concept of an *executor*.  An executor is an object that manages processes, threads, greenthreads, or other workers.  It contains both the functionality of pools and functionality similar to that of standalone `Process` or `Thread` objects.  They are initalized very similarly to a `Pool` object, in that they take as an argument the number of workers you want the executor to manage.  Executors offer two methods that make parallelism easy.  First, they have a `map` method which behaves almost exactly the same as that of `Pool` objects.  So already you might as well be using them over pools since they, at the minimum, are just as easy to use.  Second they have a `submit` method.  This method is sort of like instantiating a worker as we did before, but instead it submits a task.  You supply the method a function and its arguments, and the executor immediately takes over everything and starts running the tasks as soon as it can.  Like pools, executors will wait until they have the resources available before starting the task.  The `submit` method returns a `future` object.  Future objects represent the task that that was submitted to the executor and offer a way of interfacing with the task while waiting for the task to complete.

### 3.2 Futures

Futures are objects representing some task that is yet to complete (or already has if you wait long enough).  Futures offer methods of checking the status of the task, retrieving the results of the task, and modifying the task.  They are very convenient objects with a simple API allowing the user to interface with results that are coming in the future.  The `concurrent.futures` module, as well as other third party Python libraries, offer interfaces for easily dealing with `future` objects to make them as convenient as possible.  You can have Python block until a future finishes running and you can retrieve the results of a collection of futures asynchronously, working on the outputs as soon as they're ready.

### 3.3 Let's Get Coding

Python 3 offers 2 native executor classes, both with very similar APIs and behaviors.  The `concurrent.futures` library contains a `ThreadPoolExecutor` class as well as a `ProcessPoolExecutor` class.  For sake of simplicity we'll just use the `ThreadPoolExecutor`.  These examples will work just as well with the `ProcessPoolExecutor`, but keep in mind that when spawning processes, memory behavior changes and complicated custom objects might not work properly.

In [121]:
from concurrent.futures import ThreadPoolExecutor

We can intialize a `ThreadPoolExecutor` exactly the same way we initialized the `Pool` above.  As an aside, executors can be used as context managers, but we'll keep to using them as regular objects for now.

In [134]:
NUM_THREADS = 10
N_VALUES = 20 

thread_pool = ThreadPoolExecutor(10)
for n, f in thread_pool.map(fibonacci, range(N_VALUES)):
    print(f'fibonacci({n})={f}')

fibonacci(0)=0
fibonacci(1)=1
fibonacci(2)=1
fibonacci(3)=2
fibonacci(4)=3
fibonacci(5)=5
fibonacci(6)=8
fibonacci(7)=13
fibonacci(8)=21
fibonacci(9)=34
fibonacci(10)=55
fibonacci(11)=89
fibonacci(12)=144
fibonacci(13)=233
fibonacci(14)=377
fibonacci(15)=610
fibonacci(16)=987
fibonacci(17)=1597
fibonacci(18)=2584
fibonacci(19)=4181


Works just the same as before!  But using `map` doesn't offer us all that much control over the tasks that we're trying to run.  Let's try it again using `submit` to get a series of `future` objects.

In [135]:
futures = [thread_pool.submit(fibonacci, n) for n in range(N_VALUES)] # This is a list of futures

To get the results of the function call, we simply use the `result` method of the `future` objects.

In [125]:
for future in futures:
    n, f = future.result()
    print(f'fibonacci({n})={f}')

fibonacci(0)=0
fibonacci(1)=1
fibonacci(2)=1
fibonacci(3)=2
fibonacci(4)=3
fibonacci(5)=5
fibonacci(6)=8
fibonacci(7)=13
fibonacci(8)=21
fibonacci(9)=34
fibonacci(10)=55
fibonacci(11)=89
fibonacci(12)=144
fibonacci(13)=233
fibonacci(14)=377
fibonacci(15)=610
fibonacci(16)=987
fibonacci(17)=1597
fibonacci(18)=2584
fibonacci(19)=4181


In some cases we may be stuck waiting for the results of the run.  Let's modify the `fibonacci` function to have some random variability in the time it takes to compute.

In [128]:
import random

def fibonacci(n):
    time.sleep(random.random()) # Slow down our process randomly
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return n, a

The `concurrent.futures` library offers a nice function for getting the results of a list of futures as they complete.  The function is appropriately named `as_completed` and returns an iterator over the futures as they complete.

In [129]:
from concurrent.futures import as_completed

futures = [thread_pool.submit(fibonacci, n) for n in range(N_VALUES)] 
for future in as_completed(futures):
    n, f = future.result()
    print(f'fibonacci({n})={f}')

fibonacci(0)=0
fibonacci(6)=8
fibonacci(7)=13
fibonacci(12)=144
fibonacci(9)=34
fibonacci(10)=55
fibonacci(8)=21
fibonacci(3)=2
fibonacci(16)=987
fibonacci(1)=1
fibonacci(11)=89
fibonacci(4)=3
fibonacci(13)=233
fibonacci(14)=377
fibonacci(17)=1597
fibonacci(5)=5
fibonacci(2)=1
fibonacci(19)=4181
fibonacci(15)=610
fibonacci(18)=2584


We see that this returned our results asynchronously, so we were able to see (and act) on our data as soon as it is ready.  This can be really nice if you want to see the progress of your parallel tasks in real time.  Often times collections of futures are instantiated as a dictionary instead of a list, with the future itself as the key and some marker as the value.  This allows you to pass the dictionary to `as_completed` and easily recover which task it was that you submitted.

We can also check the status of `future` objects as they run.  We can look at whether or not they have completed, if they're running, if they've been canceled, or if they ran into errors.

In [136]:
futures = [thread_pool.submit(fibonacci, n) for n in range(N_VALUES)] 
for future in futures:
    print(future, ':', sep='', end=' ')
    print('Completed?', future.done(), end=' ')
    print('Running?', future.running())
    
time.sleep(0.2)
print("\n  Wait a little bit...\n")

for future in futures:
    print(future, ':', sep='', end=' ')
    print('Completed?', future.done(), end=' ')
    print('Running?', future.running())

<Future at 0x7f39142d6e80 state=pending>: Completed? False Running? True
<Future at 0x7f39142d64e0 state=running>: Completed? False Running? True
<Future at 0x7f39142d6d68 state=running>: Completed? False Running? True
<Future at 0x7f39142d6518 state=running>: Completed? False Running? True
<Future at 0x7f39142d6cf8 state=running>: Completed? False Running? True
<Future at 0x7f39142d69e8 state=running>: Completed? False Running? True
<Future at 0x7f39142d6390 state=running>: Completed? False Running? True
<Future at 0x7f39142d6940 state=running>: Completed? False Running? True
<Future at 0x7f39142d6588 state=running>: Completed? False Running? True
<Future at 0x7f39142d6668 state=running>: Completed? False Running? True
<Future at 0x7f39142d63c8 state=running>: Completed? False Running? True
<Future at 0x7f39142d6358 state=pending>: Completed? False Running? False
<Future at 0x7f39142d68d0 state=pending>: Completed? False Running? False
<Future at 0x7f39142d6860 state=pending>: Complet

The `concurrent.futures` module offers a really simple and easy to use API for parallelism, and becomes the basis for the distributed computing interface for Dask, which we will discuss in the next notebook.